In [203]:
from IPython.core.display import display, HTML, Markdown
from tabulate import tabulate
import pandas as pd

In [188]:
def evaluation_format(answ, pt_green,pt_red):
    pt_blue=0
    if pt_green!=0:
        pt_blue=pt_red-pt_green
        pt_red=0
    return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>, \
                                    <span style='color:blue'>[{pt_blue} possible pt]</span>, \
                                    <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

def check_num_paths_to_goal(env, num_paths, row, col):
    for i in range (0,row-1):
        if num_paths[i][0]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(1,1)$ e pertanto $num\_paths\_to[${i}$][0] = 0$"
    for j in range (0,col-1):
        if num_paths[0][j]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(1,1)$ e pertanto $num\_paths\_to[0][${j}$] = 0$"
    num_paths_forgiving = num_paths.copy()
    num_paths_forgiving[1][1] = 1
    for i in range(row-1,0,-1):
        for j in range (col-1,0,-1):
            if i==1 and j==1:
                return  evaluation_format("Si", 1, 10)+"Non riscontro particolari problemi della tua versione della matrice $num\_paths\_to$."
            if env[i][j]!=1:
                if env[i-1][j]!=1 and env[i][j-1]!=1:
                    if num_paths_forgiving[i][j]!=num_paths_forgiving[i-1][j]+num_paths_forgiving[i][j-1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif env[i-1][j]!=1:
                     if num_paths_forgiving[i][j]!=num_paths_forgiving[i-1][j]:
                            return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif env[i][j-1]!=1:
                    if num_paths_forgiving[i][j]!=num_paths_forgiving[i][j-1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                else:
                    if num_paths_forgiving[i][j]!=0:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
            else:
                if num_paths_forgiving[i][j]!=0:
                    return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."


def check_num_paths_to_start(env, num_paths, row, col):
    for i in range (0,row-1):
        if num_paths[i][col-1]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(8,9)$ e pertanto $num\_paths\_to[${i}$][10] = 0$"
    for j in range (0,col-1):
        if num_paths[row-1][j]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(8,9)$ e pertanto $num\_paths\_to[9][${j}$] = 0$"
    num_paths_forgiving = num_paths.copy()
    num_paths_forgiving[row-2][col-2] = 1
    for i in range(1,row-3):
        for j in range (1,col-3):
            if env[i][j]!=1:
                if env[i+1][j]!=1 and env[i][j+1]!=1:
                    if num_paths_forgiving[i][j]!=num_paths_forgiving[i+1][j]+num_paths_forgiving[i][j+1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif env[i+1][j]!=1:
                     if num_paths_forgiving[i][j]!=num_paths_forgiving[i+1][j]:
                            return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif env[i][j+1]!=1:
                    if num_paths_forgiving[i][j]!=num_paths_forgiving[i][j+1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                else:
                    if num_paths_forgiving[i][j]!=0:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
            else:
                if num_paths_forgiving[i][j]!=0:
                    return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
    for i in range (1, row-2):
        if env[i][col-2]!=1:
            if env[i+1][col-2]!=1:
                if num_paths_forgiving[i][col-2]!=num_paths_forgiving[i+1][col-2]+num_paths_forgiving[i][col-2+1]:    
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$." 
            else:
                if num_paths_forgiving[i][col-2]!=0:
                    return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
    for j in range (1, col-2):
        if env[row-2][j]!=1:
            if env[row-2][j+1]!=1:
                if num_paths_forgiving[row-2][j]!=num_paths_forgiving[row-2+1][j]+num_paths_forgiving[row-2][j+1]:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$." 
            else:
                if num_paths_forgiving[row-2][j]!=0:
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
    return  evaluation_format("Si", 1, 10)+"Non riscontro particolari problemi della tua versione della matrice $num\_paths\_to$."

## Esercizio \[60 pts\]
(campo minato) Ricerca di cammini in una griglia rettangolare con celle proibite.

Si consideri la seguente tabella di numeri interi.

In [204]:
m,n = 8,9
table = [
         [0, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
         [0, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ,1 ],
         [0, 0 , 0 , 0 , 0 ,1 ,1 , 0 , 0 , 0 ],
         [0, 0 ,1 , 0 ,1 , 0 , 0 , 0 , 0 , 0 ],
         [0, 0 , 0 , 0 , 0 , 0 , 0 , 0 ,1 , 0 ],
         [0, 0 , 0 ,1 , 0 ,1 , 0 , 0 , 0 , 0 ],
         [0, 0 , 0 , 0 , 0 , 0 , 0 ,1 , 0 , 0 ],
         [0, 0 ,1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
         [0, 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ]
        ]

In [205]:
columns=["1","2","3","4","5","6","7","8","9"]
index=pd.Index(["0","A","B","C","D","E","F","G","H"])
df=pd.DataFrame(table,index=index)
df = df.iloc[1:,1:]

print(tabulate(df, headers=columns, tablefmt='fancy_grid'))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │   7 │   8 │   9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │   0 │   0 │   0 │   0 │   1 │   1 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │   0 │   1 │   0 │   1 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   1 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │   0 │   0 │   1 │   0 │   1 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │   0 │   0 │   0 │   0 │   0 │   0 │   1 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   0 │   1 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼

Un robot, inizialmente situato nella cella $A1=(1,1)$, deve portarsi nella cella $H9=(8,9)$.
Le celle che riportano il numero 1 contengono una mina od altre trapole mortali, ed il robot deve evitarle. 
I movimenti base possibili sono il passo verso destra (ad esempio il primo passo potrebbe avvenire dalla cella $A1$ alla cella $A2$) ed il passo verso il basso (ad esempio, come unica altra alternativa per il primo passo il robot potrebbe posrtarsi quindi nella cella $B1$).\
Quanti sono i possibili percorsi che può fare il robot per andare dalla cella $A1$ alla cella $H9$?\

__Richieste__:

1. __\[10 pts\]__ A mano o tramite un programma componi la matrice $num\_paths\_to$ di dimensione $(m+1)\times(n+1)$, nella cui cella $num\_paths\_to[i][j]$, per ogni $i = 0,..., m+1$ e $j = 0,..., n+1$, sia riposto il numero di cammini dalla cella $A1=(1,1)$ alla generica cella $(i,j)$.

In [29]:
num_paths_to_goal= [ [0]*(n+1) for _ in range(m+1) ]

num_paths_to_goal= [
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 1, 1, 1, 1, 1, 1, 1, 1, 0], 
                    [0, 1, 2, 3, 4, 0, 0, 1, 2, 2], 
                    [0, 1, 0, 3, 0, 0, 0, 1, 3, 5],
                    [0, 1, 1, 4, 4, 4, 4, 5, 0, 5], 
                    [0, 1, 2, 0, 4, 0, 4, 9, 9, 14], 
                    [0, 1, 3, 3, 7, 7, 11, 0, 9, 23], 
                    [0, 1, 0, 3, 10, 17, 28, 28, 37, 60], 
                    [0, 1, 1, 4, 14, 31, 59, 87, 124, 184]
                   ]


La tua versione attuale della matrice $num\_paths\_to$ è la seguente:

In [206]:
df2=pd.DataFrame(num_paths_to_goal,index=index)
df2 = df2.iloc[1:,1:]
print(tabulate(df2, headers=columns, tablefmt='fancy_grid'))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │   7 │   8 │   9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │   1 │   1 │   1 │   1 │   1 │   1 │   1 │   1 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │   1 │   2 │   3 │   4 │   0 │   0 │   1 │   2 │   2 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │   1 │   0 │   3 │   0 │   0 │   0 │   1 │   3 │   5 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │   1 │   1 │   4 │   4 │   4 │   4 │   5 │   0 │   5 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │   1 │   2 │   0 │   4 │   0 │   4 │   9 │   9 │  14 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │   1 │   3 │   3 │   7 │   7 │  11 │   0 │   9 │  23 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   1 │   0 │   3 │  10 │  17 │  28 │  28 │  37 │  60 │
├────┼─────┼─────┼─────┼

In [207]:
display(Markdown(check_num_paths_to_goal(table,num_paths_to_goal,m+1,n+1)))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>Non riscontro particolari problemi della tua versione della matrice $num\_paths\_to$.

1. __\[10 pts\]__ Componi ora una matrice $num\_paths\_from$, di dimensione $(m+2)times(n+2)$, nella cui cella $num\_paths\_from[i][j]$, per ogni $i = 1,..., m+1$ e $j = 1,..., n+1$, sia riposto il numero di cammini dalla generica cella $(i,j)$ alla cella $H9=(8,9)$.

In [208]:
num_paths_to_start= [ [0]*(n+2) for _ in range(m+2) ]
num_paths_to_start=[
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 184, 63, 36, 9, 9, 9, 9, 2, 0, 0], 
                    [0, 121, 27, 27, 0, 0, 0, 7, 2, 1, 0], 
                    [0, 94, 0, 27, 0, 29, 17, 5, 1, 1, 0], 
                    [0, 94, 49, 27, 27, 12, 12, 4, 0, 1, 0], 
                    [0, 45, 22, 0, 15, 0, 8, 4, 4, 1, 0], 
                    [0, 23, 22, 22, 15, 9, 4, 0, 3, 1, 0], 
                    [0, 1, 0, 7, 6, 5, 4, 3, 2, 1, 0], 
                    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                   ]


La tua versione attuale della matrice $num\_paths\_to$ è la seguente:

In [200]:
index=pd.Index(["0","A","B","C","D","E","F","G","H","0"])
df3=pd.DataFrame(num_paths_to_start,index=index)
df3 = df3.iloc[1:-1,1:-1]
print(tabulate(df3, headers=columns, tablefmt='fancy_grid'))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │   7 │   8 │   9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │ 184 │  63 │  36 │   9 │   9 │   9 │   9 │   2 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │ 121 │  27 │  27 │   0 │   0 │   0 │   7 │   2 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │  94 │   0 │  27 │   0 │  29 │  17 │   5 │   1 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │  94 │  49 │  27 │  27 │  12 │  12 │   4 │   0 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │  45 │  22 │   0 │  15 │   0 │   8 │   4 │   4 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │  23 │  22 │  22 │  15 │   9 │   4 │   0 │   3 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   1 │   0 │   7 │   6 │   5 │   4 │   3 │   2 │   1 │
├────┼─────┼─────┼─────┼

In [201]:
display(Markdown(check_num_paths_to_start(table, num_paths_to_start,m+2,n+2)))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>Non riscontro particolari problemi della tua versione della matrice $num\_paths\_to$.

1. __\[10 pts\]__ [Esercizio senza monete] Quanti sono i percorsi possibili se la partenza è in A–1?

184

2. __\[10 pts\]__ E se la partenza è in B–3?

27

3. __\[10 pts\]__ E se con partenza in A–1 il robot deve giungere in F–6?

11

4. __\[10 pts\]__ E se con partenza in A–1 ed arrivo in H–9 al robot viene richiesto di passare per
la cella D–5?

48